# Experiment 002: C++ SA Optimizer

Test the C++ SA optimizer from Jonathan Chan's kernel on our current best solution.
The strategy notes that previous experiments showed 0 improvement, but we should verify this.

In [1]:
import pandas as pd
import numpy as np
import os
import shutil

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

from shapely.geometry import Polygon

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_bounding_side(polygons):
    if not polygons:
        return 0
    all_points = []
    for poly in polygons:
        all_points.extend(list(poly.exterior.coords))
    all_points = np.array(all_points)
    min_x, min_y = all_points.min(axis=0)
    max_x, max_y = all_points.max(axis=0)
    return max(max_x - min_x, max_y - min_y)

def compute_score_for_n(df, n):
    prefix = f"{n:03d}_"
    trees = df[df['id'].str.startswith(prefix)]
    if len(trees) != n:
        return float('inf')
    polygons = [create_tree_polygon(parse_value(row['x']), parse_value(row['y']), parse_value(row['deg'])) for _, row in trees.iterrows()]
    side = compute_bounding_side(polygons)
    return side**2 / n

def compute_total_score(df):
    total = 0
    for n in range(1, 201):
        total += compute_score_for_n(df, n)
    return total

print("Functions defined")

Functions defined


In [2]:
# Load current best solution
df_best = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
original_score = compute_total_score(df_best)
print(f"Original score: {original_score:.6f}")

Original score: 70.659959


In [3]:
# Check if bbox3 binary exists and is executable
bbox3_path = '/home/code/external_data/saspav/bbox3'
if os.path.exists(bbox3_path):
    os.chmod(bbox3_path, 0o755)
    print(f"bbox3 binary found at {bbox3_path}")
    !{bbox3_path} --help 2>&1 | head -20
else:
    print("bbox3 binary not found")

bbox3 binary found at /home/code/external_data/saspav/bbox3
/home/code/external_data/saspav/bbox3: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.34' not found (required by /home/code/external_data/saspav/bbox3)


In [ ]:
# Copy the best solution to working directory
shutil.copy('/home/code/external_data/saspav/santa-2025.csv', '/home/code/experiments/002_cpp_optimizer/input.csv')
print("Copied input file")

In [ ]:
# Run bbox3 optimizer with limited iterations to test
# -n: number of iterations, -r: number of restarts
import subprocess
import time

start_time = time.time()
result = subprocess.run(
    [bbox3_path, '-n', '1000', '-r', '4'],
    cwd='/home/code/experiments/002_cpp_optimizer',
    capture_output=True,
    text=True,
    timeout=300  # 5 minute timeout
)
end_time = time.time()

print(f"Execution time: {end_time - start_time:.1f}s")
print("STDOUT:")
print(result.stdout[:2000] if result.stdout else "(empty)")
print("\nSTDERR:")
print(result.stderr[:1000] if result.stderr else "(empty)")

In [ ]:
# Check if output was generated
output_files = [f for f in os.listdir('/home/code/experiments/002_cpp_optimizer') if f.endswith('.csv')]
print(f"CSV files in directory: {output_files}")

# Check for any improvement
for f in output_files:
    if f != 'input.csv':
        df_opt = pd.read_csv(f'/home/code/experiments/002_cpp_optimizer/{f}')
        opt_score = compute_total_score(df_opt)
        improvement = original_score - opt_score
        print(f"{f}: score={opt_score:.6f}, improvement={improvement:.9f}")

In [4]:
# bbox3 binary is incompatible with this system's GLIBC
# Let's try to compile the C++ optimizer from Jonathan Chan's kernel

# First, let's extract the C++ code from the notebook
import json

with open('/home/code/research/kernels/jonathanchan_santa25-ensemble-sa-fractional-translation/santa25-ensemble-sa-fractional-translation.ipynb', 'r') as f:
    nb = json.load(f)

# Find the cell with C++ code
for cell in nb['cells']:
    if cell['cell_type'] == 'code':
        source = ''.join(cell['source'])
        if '#include' in source and 'fractional_translation' in source:
            # Save the C++ code
            with open('/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp', 'w') as cpp_file:
                cpp_file.write(source)
            print("Extracted C++ code")
            print(f"Code length: {len(source)} characters")
            break

Extracted C++ code
Code length: 23005 characters


In [5]:
# Compile the C++ optimizer
!g++ -O3 -march=native -std=c++17 -fopenmp -o /home/code/experiments/002_cpp_optimizer/sa_v1_parallel /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp 2>&1

/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:1:1: error: expected unqualified-id before ‘%’ token
    1 | %%writefile a.cpp
      | ^
In file included from /usr/include/c++/9/cmath:43,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:41,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/include/c++/9/ext/type_traits.h:162:35: error: ‘bool __gnu_cxx::__is_null_pointer’ redeclared as different kind of entity
  162 |   __is_null_pointer(std::nullptr_t)
      |                                   ^
/usr/include/c++/9/ext/type_traits.h:157:5: note: previous declaration ‘template<class _Type> bool __gnu_cxx::__is_null_pointer(_Type)’
  157 |     __is_null_pointer(_Type)
      |     ^~~~~~~~~~~~~~~~~
/usr/include/c++/9/ext/type_traits.h:162:26: error: ‘nullptr_t’ is not a member of ‘std’
  162 |   __is_null_pointer(std::nullptr_t)
      |                          ^~~~~~~~~
In file included from /usr/include/c++/9/bits

/usr/include/c++/9/type_traits:1246:37: error: ‘size_t’ is not a member of ‘std’; did you mean ‘size_t’?
 1246 |     : public integral_constant<std::size_t, alignof(_Tp)> { };
      |                                     ^~~~~~
In file included from /usr/include/stdlib.h:31,
                 from /usr/include/c++/9/bits/std_abs.h:38,
                 from /usr/include/c++/9/cmath:47,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:41,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/lib/gcc/x86_64-linux-gnu/9/include/stddef.h:209:23: note: ‘size_t’ declared here
  209 | typedef __SIZE_TYPE__ size_t;
      |                       ^~~~~~
In file included from /usr/include/c++/9/bits/move.h:55,
                 from /usr/include/c++/9/bits/stl_pair.h:59,
                 from /usr/include/c++/9/bits/stl_algobase.h:64,
                 from /usr/include/c++/9/bits/specfun.h:45,
                 from /usr/include/c++/9/cmath:19

on before ‘const’
  141 | _GLIBCXX_NODISCARD void* operator new[](std::size_t, const std::nothrow_t&) _GLIBCXX_USE_NOEXCEPT
      |                                                      ^~~~~
/usr/include/c++/9/new:148:44: error: declaration of ‘operator new’ as non-function
  148 | _GLIBCXX_NODISCARD void* operator new(std::size_t, std::align_val_t)
      |                                            ^~~~~~
/usr/include/c++/9/new:148:44: error: ‘size_t’ is not a member of ‘std’; did you mean ‘size_t’?
In file included from /usr/include/stdlib.h:31,
                 from /usr/include/c++/9/bits/std_abs.h:38,
                 from /usr/include/c++/9/cmath:47,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:41,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/lib/gcc/x86_64-linux-gnu/9/include/stddef.h:209:23: note: ‘size_t’ declared here
  209 | typedef __SIZE_TYPE__ size_t;
      |                       ^~~~~~
In file inclu

raits, _Alloc>::basic_string() [with _CharT = char32_t; _Traits = std::char_traits<char32_t>; _Alloc = std::allocator<char32_t>]’
  435 |       basic_string()
      |       ^~~~~~~~~~~~
/usr/include/c++/9/bits/basic_string.h:435:7: note:   candidate expects 0 arguments, 2 provided
/usr/include/c++/9/bits/basic_string.h:149:7: note: candidate: ‘std::__cxx11::basic_string<_CharT, _Traits, _Alloc>::basic_string(std::__cxx11::basic_string<_CharT, _Traits, _Alloc>::__sv_wrapper, const _Alloc&) [with _CharT = char32_t; _Traits = std::char_traits<char32_t>; _Alloc = std::allocator<char32_t>]’
  149 |       basic_string(__sv_wrapper __svw, const _Alloc& __a)
      |       ^~~~~~~~~~~~
/usr/include/c++/9/bits/basic_string.h:149:33: note:   no known conversion for argument 1 from ‘const char32_t*’ to ‘std::__cxx11::basic_string<char32_t>::__sv_wrapper’
  149 |       basic_string(__sv_wrapper __svw, const _Alloc& __a)
      |                    ~~~~~~~~~~~~~^~~~~
In file included from /usr/includ

/usr/include/c++/9/utility:114:54: error: ‘__i’ was not declared in this scope
  114 |     using __tuple_element_t = typename tuple_element<__i, _Tp>::type;
      |                                                      ^~~
/usr/include/c++/9/utility:114:62: error: template argument 1 is invalid
  114 |     using __tuple_element_t = typename tuple_element<__i, _Tp>::type;
      |                                                              ^
/usr/include/c++/9/utility:116:12: error: ‘std::size_t’ has not been declared
  116 |   template<std::size_t __i, typename _Tp>
      |            ^~~
/usr/include/c++/9/utility:117:26: error: ‘__i’ was not declared in this scope
  117 |     struct tuple_element<__i, const _Tp>
      |                          ^~~
/usr/include/c++/9/utility:117:40: error: template argument 1 is invalid
  117 |     struct tuple_element<__i, const _Tp>
      |                                        ^
/usr/include/c++/9/utility:122:12: error: ‘std::size_t’ has not been 

...

In file included from /usr/include/c++/9/filesystem:37,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:129,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/include/c++/9/bits/fs_path.h: In static member function ‘static std::filesystem::__cxx11::path::string_type std::filesystem::__cxx11::path::_S_convert_loc(_Iter, _Iter, const std::locale&)’:
/usr/include/c++/9/bits/fs_path.h:534:57: error: ‘const string’ {aka ‘const class std::__cxx11::basic_string<char>’} has no member named ‘size’
  534 |  return _S_convert_loc(__str.data(), __str.data()+__str.size(), __loc);
      |                                                         ^~~~
/usr/include/c++/9/bits/fs_path.h: In static member function ‘static std::filesystem::__cxx11::path::string_type std::filesystem::__cxx11::path::_S_convert_loc(_InputIterator, std::filesystem::__cxx11::path::__null_terminated, const std::locale&)’:
/usr/include/c++/9/bits/fs_path.h:543:53: er

/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp: In member function ‘std::vector<int> Cfg::findCornerTrees() const’:
/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:136:14: error: ‘std::tuple_size<std::tuple<double, double, double, double>>::value’ is not an integral constant expression
  136 |         auto [gx0, gy0, gx1, gy1] = getBBox();
      |              ^~~~~~~~~~~~~~~~~~~~
/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp: In function ‘Cfg sa_v3(Cfg, int, double, double, double, double, uint64_t)’:
/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:165:18: error: ‘std::tuple_size<std::pair<double, double>>::value’ is not an integral constant expression
  165 |             auto [cx, cy] = cur.centroid();
      |                  ^~~~~~~~
/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:172:43: error: ‘dy’ was not declared in this scope; did you mean ‘d’?
  172 |                 double d = sqrt(dx * dx + dy * dy);
      |    

In file included from /usr/include/c++/9/exception:143,
                 from /usr/include/c++/9/ios:39,
                 from /usr/include/c++/9/istream:38,
                 from /usr/include/c++/9/sstream:38,
                 from /usr/include/c++/9/complex:45,
                 from /usr/include/c++/9/ccomplex:39,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:54,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/include/c++/9/bits/exception_ptr.h: In instantiation of ‘std::__exception_ptr::exception_ptr std::make_exception_ptr(_Ex) [with _Ex = std::future_error]’:
/usr/include/c++/9/future:438:68:   required from here
/usr/include/c++/9/bits/exception_ptr.h:188:11: error: no matching function for call to ‘operator new(sizetype, void*&)’
  188 |           ::new (__e) _Ex(__ex);
      |           ^~~~~~~~~~~~~~~~~~~~~
<built-in>: note: candidate: ‘void* operator new(long unsigned int)’
<built-in>: note:   candidate expec

/usr/include/c++/9/bits/basic_string.h: In instantiation of ‘void std::__cxx11::basic_string<_CharT, _Traits, _Alloc>::_M_dispose() [with _CharT = char32_t; _Traits = std::char_traits<char32_t>; _Alloc = std::allocator<char32_t>]’:
/usr/include/c++/9/bits/basic_string.h:662:9:   required from ‘std::__cxx11::basic_string<_CharT, _Traits, _Alloc>::~basic_string() [with _CharT = char32_t; _Traits = std::char_traits<char32_t>; _Alloc = std::allocator<char32_t>]’
/usr/include/c++/9/bits/fs_path.h:1056:53:   required from here
/usr/include/c++/9/bits/basic_string.h:236:15: error: using invalid field ‘std::__cxx11::basic_string<_CharT, _Traits, _Alloc>::<unnamed union>::_M_allocated_capacity’
In file included from /usr/include/c++/9/bits/locale_conv.h:41,
                 from /usr/include/c++/9/locale:43,
                 from /usr/include/c++/9/iomanip:43,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:72,
                 from /home/code/experiments/002_cpp_optimiz

                 from /usr/include/c++/9/ios:42,
                 from /usr/include/c++/9/istream:38,
                 from /usr/include/c++/9/sstream:38,
                 from /usr/include/c++/9/complex:45,
                 from /usr/include/c++/9/ccomplex:39,
                 from /usr/include/x86_64-linux-gnu/c++/9/bits/stdc++.h:54,
                 from /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp:7:
/usr/include/c++/9/bits/basic_string.h:6023:5: note: candidate: ‘template<class _CharT, class _Traits, class _Alloc> std::__cxx11::basic_string<_CharT, _Traits, _Allocator> std::operator+(const std::__cxx11::basic_string<_CharT, _Traits, _Allocator>&, const std::__cxx11::basic_string<_CharT, _Traits, _Allocator>&)’
 6023 |     operator+(const basic_string<_CharT, _Traits, _Alloc>& __lhs,
      |     ^~~~~~~~
/usr/include/c++/9/bits/basic_string.h:6023:5: note:   template argument deduction/substitution failed:
In file included from /usr/include/c++/9/algorithm:62,
      

In [6]:
# The C++ code has Jupyter magic commands that need to be removed
# Let's read and fix the file
with open('/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp', 'r') as f:
    cpp_code = f.read()

# Remove the %%writefile magic command
if cpp_code.startswith('%%writefile'):
    # Find the first newline and skip the magic line
    cpp_code = cpp_code[cpp_code.find('\n')+1:]

# Save the fixed code
with open('/home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp', 'w') as f:
    f.write(cpp_code)

print("Fixed C++ code")
print(f"First 200 chars: {cpp_code[:200]}")

Fixed C++ code
First 200 chars: // Tree Packer v3 - C++ version matching optimizer_v3.py
// Features: Corner tree targeting, population-based search, basin hopping
// Compile: g++ -O3 -march=native -std=c++17 -o tree_packer_v3 tree_


In [7]:
# Try compiling again
!g++ -O3 -march=native -std=c++17 -fopenmp -o /home/code/experiments/002_cpp_optimizer/sa_v1_parallel /home/code/experiments/002_cpp_optimizer/sa_v1_parallel.cpp 2>&1 | head -50

In [8]:
# Check if the binary was created
import os
binary_path = '/home/code/experiments/002_cpp_optimizer/sa_v1_parallel'
if os.path.exists(binary_path):
    print(f"Binary created: {os.path.getsize(binary_path)} bytes")
    os.chmod(binary_path, 0o755)
else:
    print("Binary not created")

Binary created: 92064 bytes


In [9]:
# Copy input file and run the optimizer
import shutil
import subprocess
import time

# Copy the best solution to working directory
shutil.copy('/home/code/external_data/saspav/santa-2025.csv', '/home/code/experiments/002_cpp_optimizer/submission_best.csv')
print("Copied input file")

# Create solutions directory
os.makedirs('/home/code/experiments/002_cpp_optimizer/solutions', exist_ok=True)

# Run the optimizer with limited iterations
start_time = time.time()
result = subprocess.run(
    ['/home/code/experiments/002_cpp_optimizer/sa_v1_parallel', '-i', '/home/code/experiments/002_cpp_optimizer/submission_best.csv', '-n', '5000', '-r', '3'],
    cwd='/home/code/experiments/002_cpp_optimizer',
    capture_output=True,
    text=True,
    timeout=600  # 10 minute timeout
)
end_time = time.time()

print(f"Execution time: {end_time - start_time:.1f}s")
print("STDOUT:")
print(result.stdout[:3000] if result.stdout else "(empty)")
print("\nSTDERR:")
print(result.stderr[:1000] if result.stderr else "(empty)")

Copied input file


Execution time: 132.8s
STDOUT:
Loading /home/code/experiments/002_cpp_optimizer/submission_best.csv...
Loaded 200 configs
Initial: 70.659959

Optimizing (v3 with fractional translation)...


Initial: 70.659959
Final:   70.659959
Improve: 0.000000 (0.00%)
Time:    132.8s
Saved submission.csv


STDERR:
(empty)
